<a href="https://colab.research.google.com/github/LarsGart/NLP-Social-Media-Mining-for-Health-Monitoring/blob/experimental/Copy_of_Task_2_Data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install demoji
!pip install transformers 
!pip install pytorch-lightning
!pip install -U spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import demoji

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text

In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLP-Social-Media-Mining-for-Health-Monitoring/Task 2/task3_training.tsv', sep = '\t')
df_eval = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLP-Social-Media-Mining-for-Health-Monitoring/Task 2/task3_validation.tsv', sep = '\t')
df_eval

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver,...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: ...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
555,331289838856835072,NaN,NaN,NaN,NaN,NaN,"okay trazodone, work your magic.",NaN,NaN
556,332907977700937731,NaN,NaN,NaN,NaN,NaN,"tomorrow, sending mummers her mother's day pos...",NaN,NaN
557,333123450254274560,NaN,NaN,NaN,NaN,NaN,@kyleecakes4 i was given like 5mg of abilify a...,NaN,NaN
558,341950988455927808,NaN,NaN,NaN,NaN,NaN,"stand down, tweeps! stand down! i did it wrong...",NaN,NaN


In [ ]:
df = df.dropna(axis=1, how='all') # drop columns that are entirely NAN
df.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,@seefisch:oral drugs for pyelonephritis:ciprof...,NaN,NaN
1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as a...,NaN,NaN
2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,@stilgarg i'm ok ty have an official diagnosis...,NaN,NaN
3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN
4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,time for my daily afternoon relaxation ritual ...,NaN,NaN


In [ ]:
df_1 = df[df['type'] == "ADR"]
df_0 = df[df['type'] != "ADR"]
df_1

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
27,342314998904786945,42.0,53.0,ADR,lost vision,cymbalta,"#cymbalta withdrawal has reached a peak, lost ...",10047522.0,vision loss
28,342314998904786945,11.0,21.0,ADR,withdrawal,cymbalta,"#cymbalta withdrawal has reached a peak, lost ...",10048010.0,withdrawal syndrome
29,342322703556038657,27.0,35.0,ADR,nauseous,cipro,i hate cipro! #antibiotic #nauseous #cf #hospi...,10028823.0,nauseous
30,342349802601844737,109.0,118.0,ADR,can't cum,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...,10021574.0,inability to orgasm
31,342349802601844737,101.0,104.0,ADR,fat,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...,10047896.0,weight gain
...,...,...,...,...,...,...,...,...,...
2239,347414282138431488,38.0,42.0,ADR,flat,zyprexa,"@anyssiakaissa depakote made me feel flat, but...",10016759.0,flat affect
2240,347414282138431488,78.0,89.0,ADR,weight gain,zyprexa,"@anyssiakaissa depakote made me feel flat, but...",10047896.0,weight gain
2242,348383854089871360,72.0,91.0,ADR,gain so much weight,zyprexa,that zyprexa really makes your vocal chords ma...,10047896.0,weight gain
2244,349181683293102080,99.0,117.0,ADR,increase my weight,zyprexa,"i'm so fine today. increasing zyprexa,my condi...",10047898.0,weight increase


In [ ]:
def clean_tweet(tweet):
    tweet = re.sub(r"#", "", tweet, 0, re.MULTILINE)
    tweet = re.sub(r"rt", "", tweet, 0, re.MULTILINE)
    # tweet = tweet.lower()
    regex_mentions = r"@\w+"
    tweet = re.sub(regex_mentions, "", tweet, 0, re.MULTILINE)
#     regex_hash = r"(#[^\s]+)"
#     tweet = re.sub(regex_hash, "", tweet, 0, re.MULTILINE)
#     hash_matched = re.finditer(
#         regex_hash, tweet['full_text'], re.MULTILINE)
    url_regex = r"(http[^\s]+)"
    tweet = re.sub(url_regex, "", tweet, 0, re.MULTILINE)
    # special_char_re = r"[^(?u)\w]+"
    # tweet = re.sub(special_char_re, " ", tweet, 0, re.MULTILINE)
    return tweet

In [ ]:
def emoji2Text(text):
    emojiTranslate = { # top 19 emojis used
    '😂': 'laughing',
    '😩': 'distraught',
    '😳': 'surprise',
    '💊': 'medication',
    '😭': 'laughing crying',
    '😒': 'disapproving',
    '😴': 'bored tired',
    '😷': 'sick',
    '😔': 'remorseful',
    '👌': 'correct',
    '👍': 'approve',
    '😫': 'weary',
    '🙌': 'celebration',
    '😏': 'smug',
    '😊': 'happy',
    '😖': 'confounded',
    '💤': 'tired',
    '😁': 'cheerful',
    '❤️': 'love'}

    for word, e in emojiTranslate.items():
        text = text.replace(word.lower(), e)
    
    text = text.lstrip().rstrip()
    
    return text

In [ ]:
def remove_emojis(text):
    # emoticons_happy = list([
    #     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    #     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    #     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    #     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    #     '<3'
    # ])
    # emoticons_sad = list([
    #     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    #     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    #     ':c', ':{', '>:\\', ';('
    # ])
    all_emoticons = [':/',':l',':-)','-.-',':::---))))','&lt;',":'(",'-_-',':)','xx','xxx',';-)',
                     ':(',':-/',':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
                     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D','d:',':-l','&lt;3',
                     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',':|','o_o',';)',
                     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',':-0','-__-',':(((',
                     '<3'':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
                     ':-[', ':-<', '=/', '>:(', ':(', '>.<', ":'-(", ":'(",  ':-c',':c', ':{',  ';(', "=\\", ">:\\", ':\\']

    emojis = list(demoji.findall(text).keys())
    clean_text = demoji.replace(text, '')

    for emo in all_emoticons:
        if (emo in clean_text):
            clean_text = clean_text.replace(emo, '')
            emojis.append(emo)

    clean_text = clean_text.replace('\n', '', -1)
    
    # using regex to remove urls
    # regex_url = r"https:(.*)[A-Za-z0-9]"
    # clean_text = re.sub(regex_url, '', clean_text, 0, re.MULTILINE)

    return clean_text #, emojis

In [ ]:
df['tweet'] = df['tweet'].map(lambda tweet: clean_tweet(remove_emojis(emoji2Text(tweet))))
df_eval['tweet'] = df_eval['tweet'].map(lambda tweet: clean_tweet(remove_emojis(emoji2Text(tweet))))

In [ ]:
df

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,:oral drugs for pyelonephritisiprofloxacin lev...,NaN,NaN
1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as a...,NaN,NaN
2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,i'm ok ty have an official diagnosis of bipol...,NaN,NaN
3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN
4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,time for my daily afternoon relaxation ritual ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2241,347921687729299456,NaN,NaN,NaN,NaN,NaN,i will admit that most people would agree zypr...,NaN,NaN
2242,348383854089871360,72.0,91.0,ADR,gain so much weight,zyprexa,that zyprexa really makes your vocal chords ma...,10047896.0,weight gain
2243,348595096352075776,NaN,NaN,NaN,NaN,zyprexa,i'm hoping that i get on the zyprexa. the gi ...,NaN,NaN
2244,349181683293102080,99.0,117.0,ADR,increase my weight,zyprexa,"i'm so fine today. increasing zyprexa,my condi...",10047898.0,weight increase


In [ ]:
df_eval

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"if avelox has hu your liver, avoid tylenol al...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro : if only more do...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
555,331289838856835072,NaN,NaN,NaN,NaN,NaN,"okay trazodone, work your magic.",NaN,NaN
556,332907977700937731,NaN,NaN,NaN,NaN,NaN,"tomorrow, sending mummers her mother's day pos...",NaN,NaN
557,333123450254274560,NaN,NaN,NaN,NaN,NaN,i was given like 5mg of abilify and i still d...,NaN,NaN
558,341950988455927808,NaN,NaN,NaN,NaN,NaN,"stand down, tweeps! stand down! i did it wrong...",NaN,NaN


In [ ]:
import spacy


In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 1.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
nlp=spacy.load('en_core_web_sm')
from spacy.training.example import Example

In [ ]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
ner=nlp.get_pipe("ner")

In [ ]:
df_ADR = df_1
TRAIN_DATA = []
LABEL = "ADR"
for index, row in df_ADR.iterrows():
    tweet, begin, end = row['tweet'], int(row['begin']), int(row['end'])

    TRAIN_DATA.append((tweet,{'entities':[(begin,end,LABEL)]}))

In [ ]:
TRAIN_DATA

[('#cymbalta withdrawal has reached a peak, lost vision and almost crashed my car from a brain zap. thanks a zillion #elililly #bigpharma',
  {'entities': [(42, 53, 'ADR')]}),
 ('#cymbalta withdrawal has reached a peak, lost vision and almost crashed my car from a brain zap. thanks a zillion #elililly #bigpharma',
  {'entities': [(11, 21, 'ADR')]}),
 ('i hate cipro! #antibiotic #nauseous #cf #hospitallife #cysticfibrosis',
  {'entities': [(27, 35, 'ADR')]}),
 ("@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own",
  {'entities': [(109, 118, 'ADR')]}),
 ("@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own",
  {'entities': [(101, 104, 'ADR')]}),
 ("@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own",
  {'entities': [(71, 84, 'ADR')]}),
 ("i've had a trazodone headache al

In [ ]:
# for _, annotations in TRAIN_DATA:
#     for ent in annotations.get("entities"):
#         ner.add_label(ent[2])

In [ ]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# Add the new label to ner
LABEL = "ADR"
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
move_names

['B-ORG',
 'B-DATE',
 'B-PERSON',
 'B-GPE',
 'B-MONEY',
 'B-CARDINAL',
 'B-NORP',
 'B-PERCENT',
 'B-WORK_OF_ART',
 'B-LOC',
 'B-TIME',
 'B-QUANTITY',
 'B-FAC',
 'B-EVENT',
 'B-ORDINAL',
 'B-PRODUCT',
 'B-LAW',
 'B-LANGUAGE',
 'I-ORG',
 'I-DATE',
 'I-PERSON',
 'I-GPE',
 'I-MONEY',
 'I-CARDINAL',
 'I-NORP',
 'I-PERCENT',
 'I-WORK_OF_ART',
 'I-LOC',
 'I-TIME',
 'I-QUANTITY',
 'I-FAC',
 'I-EVENT',
 'I-ORDINAL',
 'I-PRODUCT',
 'I-LAW',
 'I-LANGUAGE',
 'L-ORG',
 'L-DATE',
 'L-PERSON',
 'L-GPE',
 'L-MONEY',
 'L-CARDINAL',
 'L-NORP',
 'L-PERCENT',
 'L-WORK_OF_ART',
 'L-LOC',
 'L-TIME',
 'L-QUANTITY',
 'L-FAC',
 'L-EVENT',
 'L-ORDINAL',
 'L-PRODUCT',
 'L-LAW',
 'L-LANGUAGE',
 'U-ORG',
 'U-DATE',
 'U-PERSON',
 'U-GPE',
 'U-MONEY',
 'U-CARDINAL',
 'U-NORP',
 'U-PERCENT',
 'U-WORK_OF_ART',
 'U-LOC',
 'U-TIME',
 'U-QUANTITY',
 'U-FAC',
 'U-EVENT',
 'U-ORDINAL',
 'U-PRODUCT',
 'U-LAW',
 'U-LANGUAGE',
 'O',
 'B-ADR',
 'I-ADR',
 'L-ADR',
 'U-ADR']

In [ ]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path


with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(70):
    
    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        LOSS = []
        texts, annotations = zip(*batch)
        
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
            
        nlp.update(
                    example,
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        LOSS.append(losses['ner'])
        # print("Losses", losses)
    print('maxLoss', max(LOSS), iteration)

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "fucking vyvanse, giving me cotton mouth. ugh." with entities "[(28, 40, 'ADR')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "worked a double shift today while dealing with eff..." with entities "[(124, 136, 'ADR')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some ent

maxLoss 1799.2664257425392 0
maxLoss 1653.352477766318 1
maxLoss 1510.739116491448 2
maxLoss 1414.5196577082108 3
maxLoss 1388.3461061499377 4
maxLoss 1323.5954067096663 5
maxLoss 1301.7330423544468 6
maxLoss 1230.1464848296102 7
maxLoss 1235.676306968 8
maxLoss 1185.6756528889691 9
maxLoss 1174.081204332681 10
maxLoss 1159.4353324026924 11
maxLoss 1112.8225288178492 12
maxLoss 1060.5664351600685 13
maxLoss 1037.9850823101856 14
maxLoss 1014.5795351190366 15
maxLoss 1004.7254037932277 16
maxLoss 957.7265802292251 17
maxLoss 990.6553689156376 18
maxLoss 985.3538293348785 19
maxLoss 939.1128325067715 20
maxLoss 924.6112087098215 21
maxLoss 914.8189553962256 22
maxLoss 918.6671798785658 23
maxLoss 905.5435558899337 24
maxLoss 902.810065707187 25
maxLoss 868.9977529508554 26
maxLoss 870.5907529387696 27
maxLoss 856.6353115228167 28
maxLoss 849.1332442975868 29
maxLoss 853.1148720229779 30
maxLoss 853.3804231571169 31
maxLoss 824.2177381409674 32
maxLoss 826.0440340790466 33
maxLoss 790.162

In [ ]:
test_text = '''do you have any medication allergies? "asthma!!!" me: "........" pt: "no wait. avelox, that\'s it!" "so no other allergies?" "right!" *cont'''
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.label_, ent.start_char, ent.end_char)

In [ ]:
df_eval_ADR = df_eval[df_eval['type'] == "ADR"]
df_eval_NonADR = df_eval[df_eval['type'] != "ADR"]

In [ ]:
df_eval_ADR = df_eval_ADR.reset_index()
df_eval_ADR

,index,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"if avelox has hu your liver, avoid tylenol al...",10024668.0,liver damage
2,2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro : if only more do...,10011906.0,death
...,...,...,...,...,...,...,...,...,...,...
360,390,342283136480206848,110.0,122.0,ADR,hypochondria,baclofen,"hmm, interesting. i may have been undergoing b...",10020965.0,hypochondriasis
361,391,342283136480206848,55.0,65.0,ADR,withdrawal,baclofen,"hmm, interesting. i may have been undergoing b...",10048010.0,withdrawal syndrome
362,392,342299048105091072,11.0,14.0,ADR,low,vyvanse,i hate the low after the high on vyvanse,10024919.0,low mood
363,393,342310086363983873,34.0,45.0,ADR,weight gain,tysabri,on a related note: tysabri causes weight gain....,10047896.0,weight gain


In [ ]:
sentencesEval = []
for index, row in df_eval.iterrows():
  test_text = row['tweet']
  doc = nlp(test_text)
  sentencesEval.append(doc.ents)
  #print('entity:',doc.ents,'| meddra_term:',row['meddra_term'])

for i in range(len(sentencesEval)):
  if len(sentencesEval[i]) > 0:
    sentencesEval[i] = 1
  else:
    sentencesEval[i] = 0

In [ ]:
evalTruth = list(df_eval['type'])

for i in range(len(evalTruth)):
  if evalTruth[i] == 'ADR':
    evalTruth[i] = 1
  else:
    evalTruth[i] = 0

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(evalTruth, sentencesEval)

array([[171,  24],
       [237, 128]])

In [ ]:
from sklearn.metrics import classification_report
target_names = ['0','1']
print(classification_report(evalTruth, sentencesEval, target_names=target_names))

              precision    recall  f1-score   support

           0       0.42      0.88      0.57       195
           1       0.84      0.35      0.50       365

    accuracy                           0.53       560
   macro avg       0.63      0.61      0.53       560
weighted avg       0.69      0.53      0.52       560



In [ ]:
sentences = []
for index, row in df_eval_ADR.iterrows():
  test_text = row['tweet']
  doc = nlp(test_text)
  sentences.append(doc.ents)
  print('entity:',doc.ents,'| meddra_term:',row['meddra_term'])

entity: () | meddra_term: drug allergy
entity: () | meddra_term: liver damage
entity: () | meddra_term: attention deficit disorder
entity: () | meddra_term: attention impaired
entity: (died,) | meddra_term: death
entity: () | meddra_term: death
entity: () | meddra_term: tendon injury
entity: (turn 'em into a bunuel movie,) | meddra_term: bizarre dreams
entity: (tendon rupture,) | meddra_term: tendon rupture
entity: () | meddra_term: withdrawal syndrome
entity: () | meddra_term: emotional distress
entity: () | meddra_term: memory loss
entity: () | meddra_term: memory impaired
entity: () | meddra_term: somnolence
entity: () | meddra_term: feeling stoned
entity: () | meddra_term: feeling stoned
entity: (druginteractions,) | meddra_term: spaced out
entity: () | meddra_term: ill feeling
entity: () | meddra_term: appetite suppressed
entity: (allergic,) | meddra_term: drug allergy
entity: () | meddra_term: withdrawal syndrome
entity: () | meddra_term: feeling hungry
entity: (muscle spasms,) |

In [ ]:
meddra_terms = {}
for index, row in df_1.iterrows():
  meddra_terms[row['meddra_term']] = int(row['meddra_code'])

In [ ]:
unusedTerms = set()
for index, row in df_eval_ADR.iterrows():
  if row['meddra_term'] not in meddra_terms:
    unusedTerms.add(row['meddra_term'])

In [ ]:
len(meddra_terms), len(unusedTerms)

(416, 61)

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 4.6 MB/s 
     |████████████████████████████████| 1.2 MB 19.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=b6692e39739bf21cc603959b006874cab229f92f2c4c8ef5c9eb7cd639db4778
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from scipy import spatial

In [ ]:
len(sentences),len(df_eval_ADR)

(365, 365)

In [ ]:
df_eval_ADR['tweet'][0]

'do you have any medication allergies? "asthma!!!" me: "........" pt: "no wait. avelox, that\'s it!" "so no other allergies?" "right!" *cont'

In [ ]:
for i in range(len(sentences)):
  if len(sentences[i]) == 0:
    sentences[i] = None
  else:
    sentences[i] = str(sentences[i][0])
sentences

[None,
 None,
 None,
 None,
 'died',
 None,
 None,
 "turn 'em into a bunuel movie",
 'tendon rupture',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'druginteractions',
 None,
 None,
 'allergic',
 None,
 None,
 'muscle spasms',
 'muscle spasms',
 'joint pain',
 None,
 None,
 None,
 None,
 None,
 None,
 'drop off crohns',
 None,
 None,
 None,
 None,
 None,
 None,
 'falling out',
 None,
 None,
 'addictive',
 'addictive',
 'addictive',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'gaining',
 'gaining',
 None,
 None,
 None,
 'drowsy',
 'drowsy',
 None,
 None,
 'detox',
 None,
 None,
 'can stay awake',
 None,
 None,
 None,
 None,
 'allergic',
 'need to sleep',
 None,
 'eat all the sweets, the sour ones seem to help with the dry mouth',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'hrs continuous sleep',
 None,
 None,
 None,
 None,
 'dream',
 'addicted',
 'feel ill',
 'feel ill',
 "won't wake up",
 None,
 'feels like i have raw nerve endings g

In [ ]:
df_eval_ADR['tweet'][i]
i

364

In [ ]:
df_eval_ADR

,index,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"if avelox has hu your liver, avoid tylenol al...",10024668.0,liver damage
2,2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro : if only more do...,10011906.0,death
...,...,...,...,...,...,...,...,...,...,...
360,390,342283136480206848,110.0,122.0,ADR,hypochondria,baclofen,"hmm, interesting. i may have been undergoing b...",10020965.0,hypochondriasis
361,391,342283136480206848,55.0,65.0,ADR,withdrawal,baclofen,"hmm, interesting. i may have been undergoing b...",10048010.0,withdrawal syndrome
362,392,342299048105091072,11.0,14.0,ADR,low,vyvanse,i hate the low after the high on vyvanse,10024919.0,low mood
363,393,342310086363983873,34.0,45.0,ADR,weight gain,tysabri,on a related note: tysabri causes weight gain....,10047896.0,weight gain


In [ ]:
for sent in sentences:
  print(type(sent),sent)

<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'str'> died
<class 'NoneType'> None
<class 'NoneType'> None
<class 'str'> turn 'em into a bunuel movie
<class 'str'> tendon rupture
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'str'> druginteractions
<class 'NoneType'> None
<class 'NoneType'> None
<class 'str'> allergic
<class 'NoneType'> None
<class 'NoneType'> None
<class 'str'> muscle spasms
<class 'str'> muscle spasms
<class 'str'> joint pain
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'str'> drop off crohns
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'str'> falling out
<class 'NoneType'> Non

In [ ]:
ground_truth = df_eval_ADR['meddra_term']
predictions = []
i = 0

for sent in sentences:
  similarity = 0
  best_meddra_term = ''
  print(sent,type(sent))
  if type(sent) == type(str):
    s = sent
    #print('string:',s)
  else:
    s = df_eval_ADR['tweet'][i]
  i+=1

  for term in meddra_terms:
    meddra_vec = sbert_model.encode([term])[0]
    sim = 1 - spatial.distance.cosine(meddra_vec, sbert_model.encode([s])[0])
    if sim > similarity:
      similarity = sim
      best_meddra_term = term
  
  print(s, '| term:',best_meddra_term,"similarity:", similarity,'\n')
  predictions.append(best_meddra_term)

None <class 'NoneType'>
do you have any medication allergies? "asthma!!!" me: "........" pt: "no wait. avelox, that's it!" "so no other allergies?" "right!" *cont | term: poisoning by antibiotic drugs similarity: 0.6194572448730469 

None <class 'NoneType'>
 if avelox has hu your liver, avoid tylenol always, as it fuher damages liver, eat grapefruit unless taking cardiac drugs | term: allergic reaction to antibiotics similarity: 0.5281740427017212 

None <class 'NoneType'>
apparently, baclofen greatly exacerbates the "ad" pa of my adhd. average length of focus today: about 30 seconds. | term: rheumatoid arthritis flare up similarity: 0.5018877983093262 

None <class 'NoneType'>
apparently, baclofen greatly exacerbates the "ad" pa of my adhd. average length of focus today: about 30 seconds. | term: rheumatoid arthritis flare up similarity: 0.5018877983093262 

died <class 'str'>
pt of mine died from cipro  :  if only more doctors thought like you! i lost my entire life to 12 cipro pills

In [ ]:
len(predictions), len(ground_truth)

(365, 365)

In [ ]:
correct = 0
for i in range(len(df_eval_ADR)):
  if predictions[i] == ground_truth[i]:
    print(predictions[i] ,ground_truth[i])
    correct+=1

print(correct/len(df_eval_ADR))

hair loss hair loss
carbohydrate craving carbohydrate craving
felt like a zombie felt like a zombie
felt like a zombie felt like a zombie
daytime sleepiness daytime sleepiness
drug withdrawal syndrome drug withdrawal syndrome
binge eating binge eating
excessive eating excessive eating
felt like a zombie felt like a zombie
felt like a zombie felt like a zombie
0.0273972602739726


In [ ]:
df_eval_ADR

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@USER__ if avelox has hu your liver, avoid tyl...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro @USER__: @USER__ i...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
390,342283136480206848,110.0,122.0,ADR,hypochondria,baclofen,"hmm, interesting. i may have been undergoing b...",10020965.0,hypochondriasis
391,342283136480206848,55.0,65.0,ADR,withdrawal,baclofen,"hmm, interesting. i may have been undergoing b...",10048010.0,withdrawal syndrome
392,342299048105091072,11.0,14.0,ADR,low,vyvanse,i hate the low after the high on vyvanse,10024919.0,low mood
393,342310086363983873,34.0,45.0,ADR,weight gain,tysabri,on a related note: tysabri causes weight gain....,10047896.0,weight gain
